In [2]:
# Import modules
import numpy as np
import numpy.linalg as LA

# Import functions
from assembly_A_local_global_matrices import *
from assembly_K_matrices import *
from assembly_B_matrices import *
from assembly_f_vectors import *
from assembly_u_solution import *
from RegularSudomainsMesh import RegularSubdomainsMesh
from utils import *
from cg import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Import data
d_dat = np.genfromtxt('data/d.dat')
fP_dat = np.genfromtxt('data/fP.dat')
fr_dat = np.genfromtxt('data/fr.dat')
Ks = np.genfromtxt('data/localK.dat')
solution = np.genfromtxt('data/solution.dat')

In [4]:
# Initial data
# Number of subdomains
Nsub_x = 4
Nsub_y = 3

# Number of remaining nodes in each subdomain
Nr_x = 4
Nr_y = 3

# Local remaining and primal indices
rs = np.array([1, 2, 4, 5, 6, 7, 9, 10])
qs = np.array([0, 3, 8, 11])
qs_left_bound = np.array([0, 8])
qs_right = np.array([3, 11])

In [5]:
# Create mesh
mesh = RegularSubdomainsMesh(Nsub_x, Nsub_y, Nr_x, Nr_y)

In [6]:
# Transformation matrices A
# Primal nodes local-global transformation matrices
APq = create_APq_matrices(mesh)

# Remaining nodes local-global transformation matrices
ARr = create_ARr_matrices(mesh)

# Dirichlet nodes local-global transformation matrices
ADd = create_ADq_matrices(mesh)

In [7]:
# Stiffness matrices K
KRR = assembly_KRR_matrix(Ks, ARr, rs, mesh)
KPP = assembly_KPP_matrix(Ks, APq, qs, qs_right, mesh)
KRP, Krqs_list = assembly_KRP_matrix(Ks, APq, ARr, qs, qs_right, rs, mesh)
KPR = KRP.T
Kqrs_list = [Krqs.T for Krqs in Krqs_list]
KPD = assembly_KPD_matrix(Ks, APq, ADd, qs_left_bound, qs_right, mesh)
KRD = assembly_KRD_matrix(Ks, ARr, ADd, qs_left_bound, rs, mesh)

In [8]:
# Assembly B matrices
BlambdaR, BRs_list = assembly_BR_matrix(mesh, ARr)

# Dirichlet boundary conditions
# Left wall remaining
BlambdaR, BRs_list = assembly_Dirichlet_BR_matrix(mesh, ARr, BlambdaR, BRs_list)

In [9]:
# Assembly d vector
d = np.zeros(mesh.Nlambda)
d[mesh.NlambdaR:] = d_dat[np.arange(len(d_dat)) % (mesh.Nr_y - 1) != 0]

In [10]:
# Assembly f vectors
# fP
fP, fD = assembly_fP_fD_vectors(mesh, fP_dat)

# fR
fR = assembly_fR_vector(mesh, fr_dat)

In [11]:
# Assembly uD vector
uD = d_dat[np.arange(len(d_dat)) % (mesh.Nr_y - 1) == 0]

In [47]:
# Matrices pre computation

In [43]:
# Matrices pre computation
KRR_inv = np.linalg.inv(KRR)

SPP = KPP - KPR @ KRR_inv @ KRP
SPP_inv = np.linalg.inv(SPP)

fPH = fP - KPD @ uD
fRH = fR - KRD @ uD

IR = np.eye(mesh.NR) # RxR identity matrix

dH = d - BlambdaR @ KRR_inv @ ((IR + KRP @ SPP_inv @ KPR @ KRR_inv) @ fRH - KRP @ SPP_inv @ fPH)
F = -BlambdaR @ KRR_inv @ (KRP @ SPP_inv @ KPR @ KRR_inv + IR) @ BlambdaR.T

In [13]:
z = np.array([0.11097952, 0.76367948, 0.3643149 , 0.43658249, 0.54056894,
       0.62960657, 0.46773485, 0.57270768, 0.02533156, 0.96696452,
       0.33969156, 0.23758129, 0.2509545 , 0.84673038, 0.435943  ,
       0.19422253, 0.61608355, 0.24816213, 0.08060656, 0.04745991,
       0.03908529, 0.70291525, 0.35247435, 0.63742649, 0.21508681,
       0.77903499, 0.27058676, 0.29763978])

In [18]:
lambdar = np.linalg.solve(F, dH)

In [19]:
# Conjugate gradient
def cg(A, b, x, tol=1e-10):
    r = b - np.dot(A, x)
    p = r.copy()
    rsold = np.dot(r.T, r)

    for i in range(len(b)):
        Ap = np.dot(A, p)
        alpha = rsold / np.dot(p.T, Ap)
        x = x + alpha * p
        r = r - alpha * Ap
        rsnew = np.dot(r.T, r)
        if np.sqrt(rsnew) < tol:
            break
        p = r + (rsnew / rsold) * p
        rsold = rsnew

    print(f"Number of iterations required: {i + 1}")
    return x

lambdacg = cg(F, dH, np.zeros_like(dH))

Number of iterations required: 14


In [34]:
res = lambdar - lambdacg
res[res < 1e-10] = 0
res

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [49]:
def subdomains_mat_vec_multiplication(p, APq, Kqrs_list, BRs_list, Krrs_inv, LA_SPP):
    # In parallel
    x = np.zeros(APq[0].shape[0])
    for (Apqs, Kqrs, Brs) in zip(APq, Kqrs_list, BRs_list):
        x += Apqs @ Kqrs @ Krrs_inv @ Brs.T @ p

    alpha = solve_cholesky(LA_SPP, x)

    n = BRs_list[0].shape[0]

    a = np.zeros(n)
    for (BRs, Kqrs, Apqs) in zip(BRs_list, Kqrs_list, APq):
        a += BRs @ Krrs_inv @ Kqrs.T @ Apqs.T @ alpha

    b = np.zeros(n)
    for BRs in BRs_list:
        b += BRs @ Krrs_inv @ BRs.T @ p

    Fp = -(a + b)
    return Fp

def cg_feti(d, lamb, tol=1e-10):
    Krrs = Ks[rs][:, rs]
    Krrs_inv = LA.inv(Krrs)
    LA_SPP = LA.cholesky(SPP)

    #r = d - np.dot(F, lamb)
    r = d - subdomains_mat_vec_multiplication(lamb, APq, Kqrs_list, BRs_list, Krrs_inv, LA_SPP)
    p = r.copy()
    rsold = np.dot(r.T, r)

    for i in range(len(d)):
        #Fp = np.dot(F, p)
        Fp = subdomains_mat_vec_multiplication(p, APq, Kqrs_list, BRs_list, Krrs_inv, LA_SPP)
        alpha = rsold / np.dot(p.T, Fp)
        lamb = lamb + alpha * p
        r = r - alpha * Fp
        rsnew = np.dot(r.T, r)
        if np.sqrt(rsnew) < tol:
            break
        p = r + (rsnew / rsold) * p
        rsold = rsnew

    print(f"Number of iterations required: {i + 1}")
    return lamb

lambdafeti = cg_feti(dH, np.zeros_like(dH))



Number of iterations required: 14


In [47]:
res = lambdacg - lambdafeti
res[res < 1e-10] = 0
res

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [45]:
#Matrix vector multiplication F*z
z = np.array([0.11097952, 0.76367948, 0.3643149 , 0.43658249, 0.54056894,
       0.62960657, 0.46773485, 0.57270768, 0.02533156, 0.96696452,
       0.33969156, 0.23758129, 0.2509545 , 0.84673038, 0.435943  ,
       0.19422253, 0.61608355, 0.24816213, 0.08060656, 0.04745991,
       0.03908529, 0.70291525, 0.35247435, 0.63742649, 0.21508681,
       0.77903499, 0.27058676, 0.29763978])

x = np.zeros(np.shape(KPR)[0])

Krrs = Ks[rs][:, rs]
Kqrs = Ks[qs][:, rs]
Krrs_inv = LA.inv(Krrs)
LA_Krrs = LA.cholesky(Krrs)
LA_SPP = LA.cholesky(SPP)
Krrs_inv = LA.inv(Krrs)

# In parallel
for (Apqs, Kqrs, Brs) in zip (APq, Kqrs_list, BRs_list):
    x += Apqs @ Kqrs @ Krrs_inv @ Brs.T @ z

alpha = solve_cholesky(LA_SPP, x)

a = np.zeros(np.shape(BlambdaR)[0])
ar = BlambdaR @ KRR_inv @ KRP @ alpha

# Sync required!
for (BRs, Kqrs, Apqs) in zip (BRs_list, Kqrs_list, APq):
    a += BRs @ Krrs_inv @ Kqrs.T @ Apqs.T @ alpha

b = np.zeros(np.shape(BlambdaR)[0])
br = BlambdaR @ KRR_inv @ BlambdaR.T @ z

for BRs in BRs_list:
    b += BRs @ Krrs_inv @ BRs.T @ z

Fz = - a - b

res = Fz - F@z
res[res < 1e-10] = 0
res

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [60]:
x = np.random.rand(dH.shape[0])
solution = conjgrad(F, dH, x, Ks, rs, qs, APq, Kqrs_list, BRs_list, BlambdaR, KRR_inv, KRP, SPP)

ValueError: operands could not be broadcast together with shapes (28,) (16,) (28,) 

In [28]:
lambdar_ = np.linalg.solve(F, dH)

In [34]:
lambda_ = cg(F, dH, np.random.rand(dH.shape[0]))

3.9870673621093533
0.4268491365454639
0.08802777248047838
0.007650723291611664
0.0011464989352788168
0.0001071037161749479
2.4653842119270505e-05
1.4816847249774737e-06
2.332701247045686e-07
1.5968470448019577e-08
7.218403380679716e-09
3.7990163905856596e-10
2.793923281472931e-11
3.1134525687140915e-12
3.7500648639272066e-14
8.41037404606813e-15
2.7703475979938214e-16
4.267719479056176e-18
4.2601743094234153e-20
4.0710649268401765e-21
Number of iterations required: 20


NameError: name 'b' is not defined

In [33]:
err = lambdar_ - lambda_
err[err < 1e-10] = 0
err

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])